In [1]:
# Colab : décommente la ligne suivante
# !pip -q install torch torchvision --index-url https://download.pytorch.org/whl/cpu
!pip -q install torchmetrics[image] torch-fidelity lpips gdown tqdm Pillow


In [1]:
# IMPORTANT: exécuter AVANT d'importer torch/numpy
import os

# Évite le crash OpenMP (duplication lib Intel)
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Limite les threads CPU (stabilise sur Windows/Anaconda)
os.environ["OMP_NUM_THREADS"]      = "1"
os.environ["MKL_NUM_THREADS"]      = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"]  = "1"

# Maintenant on peut importer en sécurité
import sys, platform, random, json, glob, shutil, math
from pathlib import Path
import numpy as np
import torch
from PIL import Image
from tqdm import tqdm

print("Python:", sys.version)
print("OS:", platform.platform())
print("Torch:", torch.__version__)
print("CUDA dispo ?", torch.cuda.is_available())


Python: 3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:20:11) [MSC v.1938 64 bit (AMD64)]
OS: Windows-11-10.0.26100-SP0
Torch: 2.5.1
CUDA dispo ? False


In [2]:
from pathlib import Path

# Racine de travail (adapte si besoin)
ROOT = Path(r"C:\Users\ilyes\Downloads\stylegan2_cond")
EXT  = ROOT / "ext"
EXT.mkdir(parents=True, exist_ok=True)

MODELS_DIR = EXT / "models"
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# Dossiers de sortie pour nos images
OUT_ROOT   = ROOT / "runs" / "eval_two_models"
OUT_ROOT.mkdir(parents=True, exist_ok=True)

print("ROOT:", ROOT)
print("EXT :", EXT)
print("MODELS_DIR:", MODELS_DIR)
print("OUT_ROOT:", OUT_ROOT)


ROOT: C:\Users\ilyes\Downloads\stylegan2_cond
EXT : C:\Users\ilyes\Downloads\stylegan2_cond\ext
MODELS_DIR: C:\Users\ilyes\Downloads\stylegan2_cond\ext\models
OUT_ROOT: C:\Users\ilyes\Downloads\stylegan2_cond\runs\eval_two_models


In [3]:
import io, zipfile, requests

sg2_dir = EXT / "stylegan2-ada-pytorch"
if not sg2_dir.exists():
    print("Téléchargement du repo StyleGAN2-ADA (ZIP)…")
    url = "https://github.com/NVlabs/stylegan2-ada-pytorch/archive/refs/heads/main.zip"
    z = requests.get(url, timeout=60).content
    with zipfile.ZipFile(io.BytesIO(z)) as zf:
        zf.extractall(EXT)
    (EXT / "stylegan2-ada-pytorch-main").rename(sg2_dir)
    print("OK:", sg2_dir)
else:
    print("Déjà présent:", sg2_dir)


Déjà présent: C:\Users\ilyes\Downloads\stylegan2_cond\ext\stylegan2-ada-pytorch


In [4]:
import requests

ffhq_url  = "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl"
ffhq_path = MODELS_DIR / "ffhq.pkl"

if not ffhq_path.exists():
    print("Téléchargement de ffhq.pkl…")
    with requests.get(ffhq_url, stream=True, timeout=60) as r:
        r.raise_for_status()
        with open(ffhq_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=1<<20):
                if chunk: f.write(chunk)
    print("OK:", ffhq_path)
else:
    print("Déjà présent:", ffhq_path)


Déjà présent: C:\Users\ilyes\Downloads\stylegan2_cond\ext\models\ffhq.pkl


In [ ]:
# === Génération StyleGAN2-ADA (FFHQ) en pur Python / CPU ===
import os, sys
from pathlib import Path
import numpy as np
from PIL import Image
from tqdm import tqdm
import torch

sg2_dir   = Path(r"C:\Users\ilyes\Downloads\stylegan2_cond\ext\stylegan2-ada-pytorch")
ffhq_path = Path(r"C:\Users\ilyes\Downloads\stylegan2_cond\ext\models\ffhq.pkl")
out_dir   = Path(r"C:\Users\ilyes\Downloads\stylegan2_cond\runs\eval_two_models\stylegan2_ffhq")
out_dir.mkdir(parents=True, exist_ok=True)

IMG_SIZE = 256
N_GEN    = 100   # tu peux augmenter ensuite

# Import des utilitaires NVIDIA
sys.path.insert(0, str(sg2_dir))
import legacy  # fourni par le repo NVLabs

device = torch.device("cpu")
with open(ffhq_path, "rb") as f:
    G_ema = legacy.load_network_pkl(f)["G_ema"].to(device).eval()

print("z_dim:", G_ema.z_dim, "| c_dim:", getattr(G_ema, "c_dim", 0))

rng = np.random.RandomState(0)
for seed in tqdm(range(N_GEN), desc="Generate (CPU)"):
    z = torch.from_numpy(rng.randn(1, G_ema.z_dim)).float().to(device)
    c = torch.zeros([1, getattr(G_ema, "c_dim", 0)], device=device)
    with torch.no_grad():
        img = G_ema(z, c, truncation_psi=1.0, noise_mode="const")
    # [-1,1] -> [0,255] HWC uint8
    img = (img * 127.5 + 128).clamp(0,255).to(torch.uint8)[0].permute(1,2,0).cpu().numpy()
    im  = Image.fromarray(img, "RGB")
    if im.size != (IMG_SIZE, IMG_SIZE):
        im = im.resize((IMG_SIZE, IMG_SIZE), Image.LANCZOS)
    im.save(out_dir / f"seed{seed:04d}.png")

print(f"OK: {N_GEN} images → {out_dir}")


z_dim: 512 | c_dim: 0


Generate (CPU):  58%|█████▊    | 58/100 [42:15:28<125:35:50, 10765.49s/it]

In [ ]:
# ==== Évaluation rapide (nécessite torchmetrics[image] et lpips) ====
import glob
import numpy as np
from PIL import Image
import torch
from torchvision import transforms as T
from torchmetrics.image.fid import FrechetInceptionDistance
from torchmetrics.image.inception import InceptionScore
import lpips

REAL_ROOT = r"C:\Users\ilyes\Downloads\stylegan2_cond\FilteredFaces"  # ton dossier d'images réelles
GEN_ROOT  = r"C:\Users\ilyes\Downloads\stylegan2_cond\runs\eval_two_models\stylegan2_ffhq"
IMG_SIZE  = 256

ToTensor256 = T.Compose([T.Resize(IMG_SIZE), T.CenterCrop(IMG_SIZE), T.ToTensor()])

def iter_folder(folder, batch=32):
    paths = sorted(glob.glob(str(Path(folder) / "*.*")))
    for i in range(0, len(paths), batch):
        imgs=[]
        for p in paths[i:i+batch]:
            im = Image.open(p).convert("RGB")
            imgs.append(ToTensor256(im))   # [0,1]
        yield torch.stack(imgs, 0)

def to_uint8(x):
    if x.min() < 0: x = (x.clamp(-1,1)+1)/2
    return (x*255.0).clamp(0,255).byte()

# FID & IS
fid = FrechetInceptionDistance(feature=2048)
isc = InceptionScore(splits=10, normalize=True)

# accumulate REAL
for real in iter_folder(REAL_ROOT, batch=32):
    fid.update((real*255).to(torch.uint8), real=True)

# accumulate FAKE
for fake in iter_folder(GEN_ROOT, batch=32):
    fid.update((fake*255).to(torch.uint8), real=False)  
    isc.update(fake)                                 

fid_score            = float(fid.compute().item())
is_mean, is_std      = isc.compute()
is_mean, is_std      = float(is_mean), float(is_std)
print(f"FID: {fid_score:.3f} | IS: {is_mean:.3f} ± {is_std:.3f}")

# LPIPS diversité
loss_fn = lpips.LPIPS(net='alex').eval()
paths = sorted(glob.glob(str(Path(GEN_ROOT)/"*.png")))
pairs = min(400, len(paths)//2)
vals=[]
for k in range(pairs):
    a,b = np.random.choice(len(paths), size=2, replace=False)
    A = ToTensor256(Image.open(paths[a]).convert("RGB")).unsqueeze(0)*2-1
    B = ToTensor256(Image.open(paths[b]).convert("RGB")).unsqueeze(0)*2-1
    with torch.no_grad():
        vals.append(float(loss_fn(A,B).item()))
print(f"LPIPS (diversité) mean={np.mean(vals):.4f} ± {np.std(vals):.4f}")
